In [7]:
import os 
import pandas as pd
from sentence_transformers import models 
from sentence_transformers import SentenceTransformer, util 
import torch 
import re
from tqdm import tqdm

In [8]:
files = os.listdir("../storage/FGH_spec_ind_claims_triplet_v0.3.1") 
len(files)

1000000

In [9]:
# index from 100000 to 105000 
query_ids, positive_ids = [], [] 

query_docs = {} 
positive_docs = {} 

for i in tqdm(range(len(files[100000:110000])), position=0, leave=True): 
    doc_ids = files[i].split("_")
    query_ids.append(doc_ids[0]) 
    positive_ids.append(doc_ids[1]) 
    query_docs[doc_ids[0]] = "" 
    positive_docs[doc_ids[1]] = "" 
    
for i in tqdm(range(len(files[100000:110000])), position=0, leave=True): 
    with open("../storage/FGH_spec_ind_claims_triplet_v0.3.1/" + files[i], "r") as f:
        data = f.read() 
    doc_ids = files[i].split("_")
    q, p, n = data.split("\n\n\n") 
    query_docs[doc_ids[0]] = q 
    positive_docs[doc_ids[1]] = p 
    
    

100%|██████████| 10000/10000 [01:52<00:00, 89.19it/s]


In [13]:
model = SentenceTransformer("simcse_DEBERTA_KFOLD1") 
df_lists = []

query_idx = 0 

limit = 6000

limit_reached = False 

for j in tqdm(range(len(positive_ids)), position=0, leave=True):  
    if limit_reached: 
        break 
    positive_doc = positive_docs[positive_ids[j]] 
    positive_doc = positive_doc.replace(".", ";") 
    p_ttl = re.search("<TTL>([\s\S]*?)<IPC>", positive_doc).group(1)
    p_ipc = re.search("<IPC>([\s\S]*?)<ABST>", positive_doc).group(1)
    p_abst = re.search("<ABST>([\s\S]*?)<CLMS>", positive_doc).group(1)
    p_clms = re.search("<CLMS>([\s\S]*?)<DESC>", positive_doc).group(1)
    p_desc = re.search("<DESC>([\s\S]*)$", positive_doc).group(1)
    
    query_doc = query_docs[query_ids[j]] 
    query_doc = query_doc.replace(".", ";") 
    q_clms = re.search("<CLMS>([\s\S]*?)<DESC>", query_doc).group(1) 
    
    splitted_query = [] 
    L = 100 # length threshold 
    for split in re.split(r'wherein|[;\n]+', q_clms.replace(".",";")): 
        if len(split) > L:
            splitted_query.append(split) 
    
    splitted_positives = [] 
    positive_start_idx = {} 
    positive_end_idx = {} 
    for split in re.split(r'wherein|[;\n]+', p_abst.replace(".",";")): 
        if len(split) > L:
            splitted_positives.append(split)
            positive_start_idx[split]=(positive_doc.find(split))
            positive_end_idx[split]=(positive_doc.find(split)+len(split))
    for split in re.split(r'wherein|[;\n]+', p_clms.replace(".",";")):
        if len(split) > L:
            splitted_positives.append(split)
            positive_start_idx[split]=(positive_doc.find(split))
            positive_end_idx[split]=(positive_doc.find(split)+len(split))
    for split in re.split(r'wherein|[;\n]+', p_desc.replace(".",";")):
        if len(split) > L:
            splitted_positives.append(split)
            positive_start_idx[split]=(positive_doc.find(split))
            positive_end_idx[split]=(positive_doc.find(split)+len(split))
    
    d = {}
    splitted_positives=list(set(splitted_positives))
    try:
        corpus_embeddings = model.encode(splitted_positives, convert_to_tensor=True, show_progress_bar=False)
    except Exception as e:
        print(e)
        continue
    
    for i in range(len(splitted_query)): 
        query = splitted_query[i] 
        query_embedding = model.encode(query, convert_to_tensor=True, show_progress_bar=False) 
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        topk=min(len(splitted_positives),10)
        top_results = torch.topk(cos_scores, k=topk) 
        
        # filtering
        if top_results[0][0] >= 0.9 or top_results[0][0] < 0.55:
            continue 
        
                
        sent_df, positive_df, score_df = [], [], [] 
        idx_num_df, label_df, query_num_df, positive_num_df = [], [], [], [] 
        positive_start_df, positive_end_df = [], [] 
        ipc_df = [] 
        
        for score, idx in zip(top_results[0], top_results[1]): 
            sent_df.append(query) 
            idx_num_df.append(j+1)
            positive_df.append(splitted_positives[idx]) 
            positive_start_df.append(positive_start_idx[splitted_positives[idx]]) 
            positive_end_df.append(positive_end_idx[splitted_positives[idx]]) 
            score_df.append("{}".format(score)) 
            label_df.append("") 
            query_num_df.append(query_ids[j]) 
            positive_num_df.append(positive_ids[j])
            ipc_df.append(p_ipc) 
        
        df = pd.DataFrame(list(zip(idx_num_df, ipc_df, sent_df, positive_df, score_df, label_df, query_num_df, 
                                   positive_num_df, positive_start_df, positive_end_df)),
                          columns=["쿼리 번호", "the active matrix drive circuit is configured ... 	Afterwards, the voltages present at the OLED ... 	0.45018550753593445 		20080231557 	20060214888 	5504 	5616
8 	4938 	G09G330 	the active matrix drive circuit is configured ... 	comparing the at least one present current-vol... 	0.430835098028183 		20080231557 	20060214888 	2202 	2369
9 	4938 	G09G330 	the active matrix drive circuit is configured ... 	The characteristic curve segment thus measure... 	0.4228767156600952 		20080231557 	20060214888 	23694 	23820IPC 분류", "쿼리 문장", "후보 문장", "유사도", "레이블", "쿼리 문서 번호", 
                                   "Positive 문서 번호", "후보 문장 start", "후보 문장 end"]) 
        df_lists.append(df) 
        
        query_idx += 1
        if query_idx >= limit: 
            limit_reached = True
            

  1%|          | 115/10000 [02:16<3:30:31,  1.28s/it]

stack expects a non-empty TensorList


  3%|▎         | 270/10000 [05:00<3:22:17,  1.25s/it] 

stack expects a non-empty TensorList


  7%|▋         | 727/10000 [14:10<2:23:51,  1.07it/s] 

...

 11%|█▏        | 1131/10000 [22:23<2:51:36,  1.16s/it] 

stack expects a non-empty TensorList


 12%|█▏        | 1184/10000 [23:34<3:00:14,  1.23s/it]

stack expects a non-empty TensorList


 16%|█▌        | 1604/10000 [31:58<2:47:22,  1.20s/it] 


In [14]:
full_df = pd.concat(df_lists) 

full_df

,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,1,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.6895131468772888,,20090118897,20060106549,19925,20103
1,1,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689429759979248,,20090118897,20060106549,11990,12271
2,1,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.6399072408676147,,20090118897,20060106549,36208,36384
3,1,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.6380811333656311,,20090118897,20060106549,36928,37071
4,1,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.5991671085357666,,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
5,1604,G09G330,the active matrix drive circuit is configured ...,One development of the method provides for mu...,0.483725368976593,,20080231557,20060214888,13748,14332
6,1604,G09G330,the active matrix drive circuit is configured ...,The feedback signal is fed to the control cir...,0.4787593483924866,,20080231557,20060214888,4642,4780
7,1604,G09G330,the active matrix drive circuit is configured ...,"Afterwards, the voltages present at the OLED ...",0.45018550753593445,,20080231557,20060214888,5504,5616
8,1604,G09G330,the active matrix drive circuit is configured ...,comparing the at least one present current-vol...,0.430835098028183,,20080231557,20060214888,2202,2369


In [18]:
full_df_v2 = full_df.drop_duplicates()

In [21]:
queries = full_df_v2['쿼리 문장'].values 

idx = 0 
query_dict = {} 

for q in queries: 
    if q in query_dict.keys(): 
        continue 
    else: 
        query_dict[q] = idx 
        idx += 1 


In [26]:
query_ids = [] 

for q in queries: 
    query_ids.append(query_dict[q]) 

In [29]:
full_df_v2['쿼리 번호'] = query_ids 

full_df_v2

/tmp/ipykernel_57/1541365336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df_v2['쿼리 번호'] = query_ids


,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,0,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.6895131468772888,,20090118897,20060106549,19925,20103
1,0,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689429759979248,,20090118897,20060106549,11990,12271
2,0,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.6399072408676147,,20090118897,20060106549,36208,36384
3,0,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.6380811333656311,,20090118897,20060106549,36928,37071
4,0,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.5991671085357666,,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
5,4938,G09G330,the active matrix drive circuit is configured ...,One development of the method provides for mu...,0.483725368976593,,20080231557,20060214888,13748,14332
6,4938,G09G330,the active matrix drive circuit is configured ...,The feedback signal is fed to the control cir...,0.4787593483924866,,20080231557,20060214888,4642,4780
7,4938,G09G330,the active matrix drive circuit is configured ...,"Afterwards, the voltages present at the OLED ...",0.45018550753593445,,20080231557,20060214888,5504,5616
8,4938,G09G330,the active matrix drive circuit is configured ...,comparing the at least one present current-vol...,0.430835098028183,,20080231557,20060214888,2202,2369


In [30]:
full_df_v2.to_csv("sentence_data_1.csv", index=False) 

Get 62 more to match 5000

In [41]:
query_ids, positive_ids = [], [] 

query_docs = {} 
positive_docs = {} 

for i in tqdm(range(len(files[200000:205000])), position=0, leave=True): 
    doc_ids = files[i].split("_")
    query_ids.append(doc_ids[0]) 
    positive_ids.append(doc_ids[1]) 
    query_docs[doc_ids[0]] = "" 
    positive_docs[doc_ids[1]] = "" 
    
for i in tqdm(range(len(files[200000:205000])), position=0, leave=True): 
    with open("../storage/FGH_spec_ind_claims_triplet_v0.3.1/" + files[i], "r") as f:
        data = f.read() 
    doc_ids = files[i].split("_")
    q, p, n = data.split("\n\n\n") 
    query_docs[doc_ids[0]] = q 
    positive_docs[doc_ids[1]] = p 
    
    

100%|██████████| 5000/5000 [00:01<00:00, 2705.29it/s]


In [42]:
model = SentenceTransformer("simcse_DEBERTA_KFOLD1") 
df_lists = []

query_idx = 0 

limit = 1000

limit_reached = False 

for j in tqdm(range(len(positive_ids)), position=0, leave=True):  
    if limit_reached: 
        break 
    positive_doc = positive_docs[positive_ids[j]] 
    positive_doc = positive_doc.replace(".", ";") 
    p_ttl = re.search("<TTL>([\s\S]*?)<IPC>", positive_doc).group(1)
    p_ipc = re.search("<IPC>([\s\S]*?)<ABST>", positive_doc).group(1)
    p_abst = re.search("<ABST>([\s\S]*?)<CLMS>", positive_doc).group(1)
    p_clms = re.search("<CLMS>([\s\S]*?)<DESC>", positive_doc).group(1)
    p_desc = re.search("<DESC>([\s\S]*)$", positive_doc).group(1)
    
    query_doc = query_docs[query_ids[j]] 
    query_doc = query_doc.replace(".", ";") 
    q_clms = re.search("<CLMS>([\s\S]*?)<DESC>", query_doc).group(1) 
    
    splitted_query = [] 
    L = 100 # length threshold 
    for split in re.split(r'wherein|[;\n]+', q_clms.replace(".",";")): 
        if len(split) > L:
            splitted_query.append(split) 
    
    splitted_positives = [] 
    positive_start_idx = {} 
    positive_end_idx = {} 
    for split in re.split(r'wherein|[;\n]+', p_abst.replace(".",";")): 
        if len(split) > L:
            splitted_positives.append(split)
            positive_start_idx[split]=(positive_doc.find(split))
            positive_end_idx[split]=(positive_doc.find(split)+len(split))
    for split in re.split(r'wherein|[;\n]+', p_clms.replace(".",";")):
        if len(split) > L:
            splitted_positives.append(split)
            positive_start_idx[split]=(positive_doc.find(split))
            positive_end_idx[split]=(positive_doc.find(split)+len(split))
    for split in re.split(r'wherein|[;\n]+', p_desc.replace(".",";")):
        if len(split) > L:
            splitted_positives.append(split)
            positive_start_idx[split]=(positive_doc.find(split))
            positive_end_idx[split]=(positive_doc.find(split)+len(split))
    
    d = {}
    splitted_positives=list(set(splitted_positives))
    try:
        corpus_embeddings = model.encode(splitted_positives, convert_to_tensor=True, show_progress_bar=False)
    except Exception as e:
        print(e)
        continue
    
    for i in range(len(splitted_query)): 
        query = splitted_query[i] 
        query_embedding = model.encode(query, convert_to_tensor=True, show_progress_bar=False) 
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        topk=min(len(splitted_positives),10)
        top_results = torch.topk(cos_scores, k=topk) 
        
        # filtering
        if top_results[0][0] >= 0.9 or top_results[0][0] < 0.55:
            continue 
        
                
        sent_df, positive_df, score_df = [], [], [] 
        idx_num_df, label_df, query_num_df, positive_num_df = [], [], [], [] 
        positive_start_df, positive_end_df = [], [] 
        ipc_df = [] 
        
        for score, idx in zip(top_results[0], top_results[1]): 
            sent_df.append(query) 
            idx_num_df.append(j+1)
            positive_df.append(splitted_positives[idx]) 
            positive_start_df.append(positive_start_idx[splitted_positives[idx]]) 
            positive_end_df.append(positive_end_idx[splitted_positives[idx]]) 
            score_df.append("{}".format(score)) 
            label_df.append("") 
            query_num_df.append(query_ids[j]) 
            positive_num_df.append(positive_ids[j])
            ipc_df.append(p_ipc) 
        
        df = pd.DataFrame(list(zip(idx_num_df, ipc_df, sent_df, positive_df, score_df, label_df, query_num_df, 
                                   positive_num_df, positive_start_df, positive_end_df)),
                          columns=["쿼리 번호", "IPC 분류", "쿼리 문장", "후보 문장", "유사도", "레이블", "쿼리 문서 번호", 
                                   "Positive 문서 번호", "후보 문장 start", "후보 문장 end"]) 
        df_lists.append(df) 
        
        query_idx += 1
        if query_idx >= limit: 
            limit_reached = True
            

  2%|▏         | 115/5000 [02:17<1:45:13,  1.29s/it]

stack expects a non-empty TensorList


  5%|▌         | 270/5000 [05:01<1:38:28,  1.25s/it]

stack expects a non-empty TensorList


  6%|▌         | 287/5000 [05:24<1:28:46,  1.13s/it]


In [43]:
full_df = pd.concat(df_lists) 

full_df

,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,1,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.6895131468772888,,20090118897,20060106549,19925,20103
1,1,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689429759979248,,20090118897,20060106549,11990,12271
2,1,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.6399072408676147,,20090118897,20060106549,36208,36384
3,1,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.6380811333656311,,20090118897,20060106549,36928,37071
4,1,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.5991671085357666,,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
5,287,G08B2900,the presence event relates to a location of t...,"By way of yet another example, user terminal 1...",0.47204747796058655,,20080235230,20070182541,46648,46914
6,287,G08B2900,the presence event relates to a location of t...,Presence Server 120 receives the presence inf...,0.47018203139305115,,20080235230,20070182541,30172,30540
7,287,G08B2900,the presence event relates to a location of t...,"In addition, the Presence Server maintains a ...",0.4690783619880676,,20080235230,20070182541,5836,6137
8,287,G08B2900,the presence event relates to a location of t...,"As is known in the art, Presence Source 106 m...",0.46806225180625916,,20080235230,20070182541,29844,30171


In [45]:
full_df_unique = full_df.drop_duplicates() 

In [46]:
full_df_unique

,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,1,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.6895131468772888,,20090118897,20060106549,19925,20103
1,1,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689429759979248,,20090118897,20060106549,11990,12271
2,1,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.6399072408676147,,20090118897,20060106549,36208,36384
3,1,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.6380811333656311,,20090118897,20060106549,36928,37071
4,1,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.5991671085357666,,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
5,287,G08B2900,the presence event relates to a location of t...,"By way of yet another example, user terminal 1...",0.47204747796058655,,20080235230,20070182541,46648,46914
6,287,G08B2900,the presence event relates to a location of t...,Presence Server 120 receives the presence inf...,0.47018203139305115,,20080235230,20070182541,30172,30540
7,287,G08B2900,the presence event relates to a location of t...,"In addition, the Presence Server maintains a ...",0.4690783619880676,,20080235230,20070182541,5836,6137
8,287,G08B2900,the presence event relates to a location of t...,"As is known in the art, Presence Source 106 m...",0.46806225180625916,,20080235230,20070182541,29844,30171


In [51]:
queries = full_df_unique['쿼리 문장'].values 

idx = 4939
query_dict = {} 

for q in queries: 
    if q in query_dict.keys(): 
        continue 
    else: 
        query_dict[q] = idx 
        idx += 1 


In [52]:
query_ids = [] 

for q in queries: 
    query_ids.append(query_dict[q]) 

full_df_unique['쿼리 번호'] = query_ids 

/tmp/ipykernel_57/674340447.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df_unique['쿼리 번호'] = query_ids


In [53]:
full_df_unique

,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,4939,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.6895131468772888,,20090118897,20060106549,19925,20103
1,4939,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689429759979248,,20090118897,20060106549,11990,12271
2,4939,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.6399072408676147,,20090118897,20060106549,36208,36384
3,4939,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.6380811333656311,,20090118897,20060106549,36928,37071
4,4939,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.5991671085357666,,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
5,5281,G08B2900,the presence event relates to a location of t...,"By way of yet another example, user terminal 1...",0.47204747796058655,,20080235230,20070182541,46648,46914
6,5281,G08B2900,the presence event relates to a location of t...,Presence Server 120 receives the presence inf...,0.47018203139305115,,20080235230,20070182541,30172,30540
7,5281,G08B2900,the presence event relates to a location of t...,"In addition, the Presence Server maintains a ...",0.4690783619880676,,20080235230,20070182541,5836,6137
8,5281,G08B2900,the presence event relates to a location of t...,"As is known in the art, Presence Source 106 m...",0.46806225180625916,,20080235230,20070182541,29844,30171


In [54]:
full_df_unique.to_csv("sentence_data_2.csv",index=False) 

In [70]:
df1 = pd.read_csv("sentence_data_1.csv") 
df2 = pd.read_csv("sentence_data_2.csv") 

df1.shape, df2.shape

((52770, 10), (8760, 10))

In [84]:
df2 = df2.iloc[:620]

In [86]:
df = pd.concat([df1, df2], axis=0) 

df

,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,0,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.689513,NaN,20090118897,20060106549,19925,20103
1,0,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689430,NaN,20090118897,20060106549,11990,12271
2,0,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.639907,NaN,20090118897,20060106549,36208,36384
3,0,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.638081,NaN,20090118897,20060106549,36928,37071
4,0,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.599167,NaN,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
615,5000,B60W3002,selecting one of the brake-based stability con...,"In an eighth embodiment of the invention, a me...",0.585400,NaN,20100211278,20080183353,21858,22564
616,5000,B60W3002,selecting one of the brake-based stability con...,"In addition to, or as an alternative to the A...",0.580420,NaN,20100211278,20080183353,79367,79666
617,5000,B60W3002,selecting one of the brake-based stability con...,"14, in an alternative variation of this embod...",0.575562,NaN,20100211278,20080183353,84286,84650
618,5000,B60W3002,selecting one of the brake-based stability con...,Considering the above drawbacks associated wit...,0.561336,NaN,20100211278,20080183353,18365,18920


In [96]:
final_df_list = [] 

for i in range(0, 5001): 
    cur_df = df[df['쿼리 번호']==i] 
    cur_df = cur_df.iloc[:10] 
    final_df_list.append(cur_df) 
    
final_df = pd.concat(final_df_list) 

final_df

,쿼리 번호,IPC 분류,쿼리 문장,후보 문장,유사도,레이블,쿼리 문서 번호,Positive 문서 번호,후보 문장 start,후보 문장 end
0,0,G06F1900,A method for forecasting the damage of compon...,"In particular, a system for predicting malfun...",0.689513,NaN,20090118897,20060106549,19925,20103
1,0,G06F1900,A method for forecasting the damage of compon...,A primary purpose of the invention is to prov...,0.689430,NaN,20090118897,20060106549,11990,12271
2,0,G06F1900,A method for forecasting the damage of compon...,The system additionally comprises means 8 ope...,0.639907,NaN,20090118897,20060106549,36208,36384
3,0,G06F1900,A method for forecasting the damage of compon...,The calculation means 8 of the system consist...,0.638081,NaN,20090118897,20060106549,36928,37071
4,0,G06F1900,A method for forecasting the damage of compon...,calculating an amount of damage caused to the ...,0.599167,NaN,20090118897,20060106549,3454,3557
...,...,...,...,...,...,...,...,...,...,...
615,5000,B60W3002,selecting one of the brake-based stability con...,"In an eighth embodiment of the invention, a me...",0.585400,NaN,20100211278,20080183353,21858,22564
616,5000,B60W3002,selecting one of the brake-based stability con...,"In addition to, or as an alternative to the A...",0.580420,NaN,20100211278,20080183353,79367,79666
617,5000,B60W3002,selecting one of the brake-based stability con...,"14, in an alternative variation of this embod...",0.575562,NaN,20100211278,20080183353,84286,84650
618,5000,B60W3002,selecting one of the brake-based stability con...,Considering the above drawbacks associated wit...,0.561336,NaN,20100211278,20080183353,18365,18920


In [103]:
final_df.to_csv("simcse_DEBERTA_KFOLD1_sentence.csv",index=False) 

In [105]:
final_df[final_df['쿼리 번호']==5000]['쿼리 문장'].values[0], final_df[final_df['쿼리 번호']==5000]['후보 문장'].values[0], 

('selecting one of the brake-based stability control subsystem, the torque-based stability control subsystem, and the drivetrain-based stability control subsystem as the dominant stability control system for the wheeled tactical vehicle based on the received three or more signals associated with current vehicle operating conditions and the received signal associated with the mobility traction control system',
 ' The present invention uses calculated or inferred road surface state information to adjust drive torque characteristics to either, depending on the sensed road condition, provide moderate front to rear and side to side torque biases to improve vehicle drivability and stability, or provide strong front to rear and side to side torque biases to improve vehicle handling driving pleasure')

In [108]:
np.max(final_df['유사도'].values)

0.8994665145874023